In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader=PyPDFDirectoryLoader("pdfs")

data=loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\nourg\AppData\Local\Temp\ipykernel_5460\1822540165.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\nourg\Documents\VS_code\QAChromaDB\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  
)

In [6]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(text_chunks))]

In [7]:
vector_store.aadd_documents(
    [t.page_content for t in text_chunks],embedding=embeddings,ids=uuids
)

<coroutine object VectorStore.aadd_documents at 0x000001EF80A99E00>

In [8]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="conversational", 
    provider="fireworks-ai",
    max_new_tokens=64
)
chat = ChatHuggingFace(llm=llm, verbose=True)

In [9]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)